### Download Elevation File According to Area Of Interest

With this code, you can download elevation data(90m) of your area of interest
from 'https://www.earthenv.org/DEM' 

In [2]:
import geopandas as gpd

In [96]:
# 'elevation_tif' .shp data is tile index data that is produced by Geoserver
# To create this data, I download all elevation data from website and
# if data includes value bigger than 0, I get that elevation data

In [3]:
world_dem_index=gpd.read_file('./shp/elevation_tif.shp')

In [62]:
world_dem_index.crs

{'init': 'epsg:4326'}

### Define Area of Interest

There are two ways define your AOI.
First, you can read your .shp or json file with geopandas.read_file method
Second, go to 'http://geojson.io/ ' and draw your aoi. Then, copy the text in left and
define as you see in below. To read with geopandas, you should us json.dumps function.

data={
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -9.16259765625,
              41.9921602333763
            ],
            [
              -1.0546875,
              41.9921602333763
            ],
            [
              -1.0546875,
              43.866218006556394
            ],
            [
              -9.16259765625,
              43.866218006556394
            ],
            [
              -9.16259765625,
              41.9921602333763
            ]
          ]
        ]
      }
    }
  ]
}

datajson=json.dumps(data) <br/> AOI=gpd.read_file(datajson)


In [ ]:
datajson=json.dumps(data)
west=gpd.read_file(datajson)


In [10]:
aoi=gpd.read_file('./shp/aoi.json')

In [63]:
aoi.crs

{'init': 'epsg:4326'}

In [97]:
# Intersect AOI and World index file

In [82]:
dem_intersect = gpd.overlay(world_dem_index, aoi, how='intersection')
dem_intersect[0:2]

,location,geometry
0,EarthEnv-DEM90_N40W005.tif,"POLYGON ((-4.999999999999996 41.9921602333763,..."
1,EarthEnv-DEM90_N40W010.tif,POLYGON ((-5.000000000001995 43.86621800655639...


In [113]:
len(dem_intersect)

2

In [ ]:
# Url should be like
#http://mirrors.iplantcollaborative.org/earthenv_dem_data/EarthEnv-DEM90/EarthEnv-DEM90_N25E010.tar.gz"
#We will create new url list. We just manipulate the end of the url.

In [111]:
main_url='http://mirrors.iplantcollaborative.org/earthenv_dem_data/EarthEnv-DEM90/'
suffix='.tar.gz'

In [116]:
img_urls_list=[]
for index,img in dem_intersect.iterrows():
    url=main_url+img['location'].split('.')[0]+suffix
    img_urls_list.append(url)
# comparing the length of the lists
len(img_urls_list)==len(dem_intersect)

True

In [133]:
import requests
import os 
import shutil

In [130]:
# Be aware end of the file path
# You should define end of the folder name with '/'
# Before start download, you can write training for loop to check the name of the files

In [136]:
# main folder of your download
target_folder='./data/'
if not os.path.isdir(target_folder):
    os.mkdir(target_folder)
# .tar.gz file location
zip_file_folder=target_folder+'elevation_zip/'
if not os.path.isdir(zip_file_folder):
    os.mkdir(zip_file_folder)
# Unzip file location
elevation_files_folder=target_folder+'elevation_files/'
if not os.path.isdir(elevation_files_folder):
    os.mkdir(elevation_files_folder)


In [138]:
%%time
for url in img_urls_list:    
    img_name=url.split('/')[-1]
    img_path=zip_file_folder+img_name
    #print(img_path)
    
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(img_path, 'wb') as f:
            f.write(response.raw.read())

    #unzip the file
    file_name=url.split('/')[-1].split('.')[0]
    file_path=elevation_files_folder+file_name    
    #print(file_path)
    shutil.unpack_archive(img_path,file_path,'gztar')
    #print('-----------')
    

CPU times: user 1.06 s, sys: 390 ms, total: 1.45 s
Wall time: 17 s
